# Newspaper Text Analysis Coding Challenge

### November 2021

In this coding challenge, your goal is to extract and analyze newspaper texts using the given input images and layout information. There will be four parts in this challenge:
1. Extract the text from the newspaper images
2. Convert the text into numerical representations
3. Analyze the text and visualize your results
4. Report your methods and findings

The dataset and problems are highly related to our on-going projects, and you will have a better sense of our team's work after finishing all the parts. 


**Instructions**: 
1. You should complete this challenge in Python. At present, we do not offer a version of the coding challenge in R, as there is relatively less support and development in R for running the deep learning models featured in this challenge, such as BERT. However, if you strongly prefer to use R for this challenge, consider checking out [this post](https://blogs.rstudio.com/ai/posts/2019-09-30-bert-r/), which is a reference for using BERT in R. 
2. All coding challenge prompts are given in this Jupyter notebook, but you can write your own functions outside of the notebook if that's preferable. We provide function skeletons in this Jupyter notebook, but please feel free to make any changes to them that work best for your code. 
3. Please provide clean and easy-to-read code. Add comments when necessary. 
4. We've designed this challenge so as to make it suitable for your local coding environments, e.g., no model training on a GPU is expected or necessary. If for some reason you find a need to train a model with a GPU, the easiest way to proceed is to upload the notebook to [Google Colab](https://colab.research.google.com/notebooks/intro.ipynb) and use the free compute resources available there. 

## Part 1: Convert Images to Text

Most historical documents (e.g., newspapers) are only available to researchers in the form of image scans. The first step in analyzing the content of a historical document is therefore to perform image-to-text conversion. This is a combination of Layout Analysis and Optical Character Recognition (OCR), whereby you parse the arrangement of "layout objects" (e.g., headlines, author bylines) in documents and turn the image data inside each layout object region into a computable text. Luckily, a team member has just finished the layout analysis for you! With the given *layout information* and *images* as inputs, you need to generate structured data in the form of full newspaper articles. Structuring the OCRed text data as newspaper articles using layout information is a prerequisite for performing further analysis on the content of historical document scans.

**Input**:

- A list of images and layout information: `List[Tuple(image_path, layout_info)]`
- The image and layout information files are stored in the `dataset/images` folder and `dataset/layout.json` file
- The `layout.json` will be stored in a format like the [COCO format](https://www.immersivelimit.com/tutorials/create-coco-annotations-from-scratch): 
    
    ```
    {
        'annotations': [
            # Specifies the rectangular boxes and types for 
            # each layout element 
            {"id": 1,        # The id of the anno object
             "bbox": [],     # (x_top_left, y_top_left, width, height) of the bounding box
             "image_id": 1,  # The associated image_id
             "category_id":2 # The category_id 
            }, ...
        ],
        'images': [
            {"id": 397133,
             "height": 427,
             "width": 640,
             "file_name": "000000397133.jpg"
            }, ...
        ],
        'categories': [
            {"supercategory": None, # We don't use this
             "id": 1,               # The category_id used for each annotation
             "name": "person"       # The name for the categories 
             }, ...
        ],
        'reading_orders': [
            # Specifies the reading order of different annotation objects 
            # The object of `end_id` comes after the object of `start_id`
            # Not all objects have the reading orders 
            {
              "from_id": 435,
              "to_id": 434,
              "image_id": 9
            }, ...
        ]
    }
    ```


**Output**:
    
- A list of parsed article information: `List[Dict(article_info)]`
    ```
    [
        {'id': 0,            # An articled id generated by you (better be continuous from 0 to len(articles))
         'headline': '',     # The headline of an article. Could be empty 
         'text': '',         # The body text of the article
         'image_id': 12,     # The id of the image where the article appears 
         'anno_ids': [31,33] # The ids of the associated layout regions 
        }, ...
    ]
    ```
    
**Steps**:

1. Iterate through the layout information.
2. Using the layout info, perform OCR on layout object regions with the open source tool TesseractOCR.
3. Join the layout object OCR texts using the reading orders available in the layout info.
4. Combine all the outputs and save in the aforementioned format.

**Notes**:

- You might need to install the [TesseractOCR engine](https://github.com/tesseract-ocr/tesseract#installing-tesseract) and its [Python wrapper](https://github.com/madmaze/pytesseract) on your computer to perform the OCR.
- You might find some functions in the [Layout Parser Library](https://layout-parser.readthedocs.io/en/latest/) helpful.
- For the reading orders:
    - The below image (or the `reading-order.png` file in the same folder as the Jupyter notebook document) shows an example of the layout objects in a full article and their reading order. 
    - A full article is usually composed of several text regions of different types. They are usually associated by several one-directional reading order pointers, illustrated by red arrows in the figure. The `from_id` is the annotation (object) id for the starting box, and the `to_id` is for the ending box. 
    - For the end of the article, there **won't** be an extra reading order point to the next title. So the reading order sequence within a full article is terminated at the last article box. 
    - Sometimes we will have reading orders for images and their captions. You can ignore these reading orders. 
    ![reading-order](reading-order.png)

In [1]:
def parse_newspaper_scan(image, layout_info):
    """
    Extract the article information from a newspaper scan 
    
    Args:
        image (numpy.ndarray): 
            The image scan for a paper 
        layout_info (dict):
            The layout information (boxes and classes + reading orders)
            for that newspaper 
    
    Returns: 
        (list(dict)):
            The parsed articles for a newspaper 
    """
    pass

## Part 2: Run BERT on the Text 

After parsing the articles from the images, we want to use natural language processing (NLP) methods to analyze the text. As texts are usually heterogeneous, we use novel deep learning models like BERT to convert them into dense numerical representations (i.e., vectors) to enable meaningful quantitative analysis. Fortunately, many BERT implementations and pre-trained model weights are open source. Thus, we don't need to spend a huge amount of time implementing the models of interest to us and training them from scratch. 

**Input**: 
- The list of articles from Part 1

**Output**:
- An $M\times N$ matrix where each row is the embedding (dense numerical representation) vector for that article 
    - $M$: len(articles)
    - $N$: The embedding dimension. 
    
**Steps**:
1. Build the BERT model and load the `bert-base-uncased` weights. 
2. For each article, use BERT to "embed" (create a dense numerical representation of) the text. 
    - You can refer to this example for creating the sentence embeddings using BERT: https://github.com/BramVanroy/bert-for-inference/blob/master/introduction-to-bert.ipynb
    - Questions to consider:
        - How do you deal with the noise (e.g., spelling mistakes, punctuation errors) in the OCRed text? Is this noise a problem for BERT?
        - How do you handle embedding very long articles? 
        - How do you synthesize token-level embeddings into sentence-level embeddings or document-level embeddings?

**Notes**:
- You might need to install and learn to use the [Transformers library](https://github.com/huggingface/transformers) and their BERT implementation and pre-trained weights.
- You can consider generating embedding vectors for just the headlines of each article, which are usually short sentences and may prove more manageable to work with. If you do so, provide a brief discussion of the tradeoffs involved in this decision in Part 4, and whether or not you would want to do this in practice.


In [2]:
def embed_newspaper_texts(articles):
    """
    Create embedding vectors for the input articles 
    
    Args:
        articles (list(dict)): 
            A list of articles 
    
    Returns: 
        (numpy.ndarray or other format):
            An MxN matrix/table for the article embeddings, where 
            each row is for the embedding vector for an article
    """
    pass

## Part 3: Visualization and Analysis 

It's hard for humans to directly interpret these dense, high-dimensional embedding vectors. Therefore, to make the results of BERT inference more transparent and intelligible, it is common to (a) perform dimensionality reduction on and (b) cluster the embedding vectors. In this part, you are required to analyze the vectors you generated in a similar way. 

**Input**:
- The embedding matrix/table from Part 2

**Output**:
- Visualization of the embedding vectors in a low-dimension space 
- Clustering results of the embedding vectors 
- Interpretation of the results, including but not limited to: 
    - Do the clusters of vectors look meaningful? 
    - Could you find clusters of important topics? 

**Steps**:
1. Choose an appropriate dimensionality reduction method and apply it to the embedding matrix. 
2. Choose an appropriate clustering method and apply it to the embedding matrix or the embedding matrix after dimensionality reduction.
3. Visualize the results using the tools that are most familiar to you and save the results.

**Notes**:
- Scikit-learn provides many handy functions for [clustering](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.cluster) and dimensionality reduction (e.g., [matrix decomposition](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.decomposition), [manifold learning methods such as t-SNE](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.manifold)).
- You can visualize the results in 2D or 3D. For drawing 3D plots, you can consider using [plotly](https://plotly.com/). 

In [3]:
def clustering_and_viz(embs):
    """
    Postprocess the embedding vectors and visualize the results 
    
    Args:
        embs (numpy.ndarray or other format):
            An MxN matrix/table for article text embeddings 
    """
    pass

## Part 4: Report Your Methods and Findings 

As a researcher, being able to explain your methods and findings clearly is very important. Therefore, we would like you to write a short report that:
1. documents your implementation and provides extra notes about your code;
2. analyzes the visualization results; and 
3. highlights any of the challenges you encountered or questions you faced in this process. 


## What to Submit

Congratulations! You've finished all of the tasks! We believe you may now have a better understanding of our work, and, hopefully, you have also learned something about deep NLP and data analysis in the process. 

To submit your completed coding challenge, you can zip all your files and send the zip file to the specified email address. Please include all the following files in your zip file: 

1. A `code` folder that has all the code (Jupyter notebooks or Python files) for generating the results:
    - Please make sure the code is runnable. 
    - Provide documentation and comments whenever necessary. 
    - You can also specify any Python libraries and versions necessary to run your code in a `requirements.txt` file. See [this post](https://note.nkmk.me/en/python-pip-install-requirements/).  
2. A `results` folder that has the intermediate results:
    - The article JSON file from Part 1 (note: you can just save the output article list as a JSON file using `json.dump`). 
    - The embedding matrix from Part 2 (note: you can consider using [`numpy.save`](https://numpy.org/doc/stable/reference/generated/numpy.save.html) to save the matrix, or use [`pandas.to_csv`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) if you have been storing the matrix as a `DataFrame`). 
    - The visualization images or HTMLs from Part 3.
3. The short report.

Thank you for completing this challenge. We will get back to you shortly! 